In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def load_data(path,target):
    df = pd.read_csv(path)
    if path == 'data/jm1.csv':
        df = df[~df.uniq_Op.str.contains("\?")]
    y = df[target]
    X = df.drop(labels = target, axis = 1)
    X = X.apply(pd.to_numeric)
    return X,y

# Cluster Driver
def cluster_driver(df,print_tree = True):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
    #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
    #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

In [14]:
def get_predicted(cluster_data_loc,metrices_loc,fold,data_location):
    train_data = pd.read_pickle(cluster_data_loc + '/train_data.pkl')
    cluster,cluster_tree,max_depth = cluster_driver(train_data)
    t_df = pd.DataFrame()
    for project in train_data.index.values.tolist():
        _s_path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + project
        s_df = prepare_data(_s_path)
        t_df = pd.concat([t_df,s_df])
    t_df.reset_index(drop=True,inplace=True)
    d = {'buggy': True, 'clean': False}
    t_df['Buggy'] = t_df['Buggy'].map(d)
    t_df, s_cols = apply_cfs(t_df)
    t_df = apply_smote(t_df)
    train_y = t_df.Buggy
    train_X = t_df.drop(labels = ['Buggy'],axis = 1)
    clf_self = LogisticRegression()
    clf_self.fit(train_X,train_y)
    test_data = pd.read_pickle(cluster_data_loc + '/test_data.pkl')
    print("Training Done")
    test_projects = test_data.index.values.tolist()
    goals = ['recall','precision','pf','pci_20','ifa']
    results = {}
    for d_project in test_projects:
        F = {}
        d_path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + d_project
        test_df = prepare_data(d_path)
        test_df.reset_index(drop=True,inplace=True)
        d = {'buggy': True, 'clean': False}
        test_df['Buggy'] = test_df['Buggy'].map(d)
        test_y = test_df.Buggy
        test_X = test_df.drop(labels = ['Buggy'],axis = 1)
        X_train, X_test, y_train, y_test = train_test_split(test_X, test_y, test_size=0.1, random_state=42)
        _test_df = pd.concat([X_test,y_test], axis = 1)
        _df_test_loc = _test_df.LOC
        _test_df = _test_df[s_cols]
        y_test = _test_df.Buggy
        X_test = _test_df.drop(labels = ['Buggy'],axis = 1)
        predicted = clf_self.predict(X_test)
        abcd = metrices.measures(y_test,predicted,_df_test_loc)
        F['precision'] = [abcd.calculate_precision()]
        F['recall'] = [abcd.calculate_recall()]
        F['pci_20'] = [abcd.get_pci_20()]
        F['ifa'] = [abcd.get_ifa()]
        F['pf'] = [abcd.get_pf()]
        for goal in goals:
            if goal == 'g':
                _goal = 'g-score'
            else:
                _goal = goal
            if _goal not in results.keys():
                results[_goal] = {}
            if d_project not in results[_goal].keys():
                results[_goal][d_project] = []
            results[_goal][d_project].append(F[_goal][0])
        for key in results:
            df = pd.DataFrame.from_dict(results[key],orient='index')
            if not Path(data_location).is_dir():
                os.makedirs(Path(data_location))
            df.to_csv(data_location + '/bellwether_' + key + '.csv')
    return results

In [15]:
for i in range(1):
    fold = str(i)
    data_location = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp_cdom_v6/fold_' + fold
    cluster_data_loc = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + fold
    metrices_loc = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    results = get_predicted(cluster_data_loc,metrices_loc,fold,data_location)
    

[cluster_id=0] N_children: 7 N_samples: 627
> [cluster_id=1] N_children: 0 N_samples: 1
> [cluster_id=2] N_children: 0 N_samples: 1
> [cluster_id=3] N_children: 5 N_samples: 55
> > [cluster_id=4] N_children: 0 N_samples: 9
> > [cluster_id=5] N_children: 0 N_samples: 9
> > [cluster_id=6] N_children: 0 N_samples: 7
> > [cluster_id=7] N_children: 0 N_samples: 17
> > [cluster_id=8] N_children: 0 N_samples: 13
> [cluster_id=9] N_children: 10 N_samples: 127
> > [cluster_id=10] N_children: 0 N_samples: 3
> > [cluster_id=11] N_children: 0 N_samples: 9
> > [cluster_id=12] N_children: 0 N_samples: 19
> > [cluster_id=13] N_children: 0 N_samples: 15
> > [cluster_id=14] N_children: 0 N_samples: 4
> > [cluster_id=15] N_children: 0 N_samples: 17
> > [cluster_id=16] N_children: 0 N_samples: 15
> > [cluster_id=17] N_children: 0 N_samples: 19
> > [cluster_id=18] N_children: 0 N_samples: 11
> > [cluster_id=19] N_children: 0 N_samples: 15
> [cluster_id=20] N_children: 16 N_samples: 183
> > [cluster_id=21]

In [16]:
results

{'recall': {'guineu.csv': [0.0],
  'campsoft.csv': [0.0],
  'gtad.csv': [0],
  'rhex.csv': [0.0],
  'owlib.csv': [0.0],
  'ktc.csv': [0.0],
  'jedit.csv': [0.0],
  'net-snmp.csv': [1.0],
  'encog-java.csv': [0.0],
  'umber.csv': [0.0],
  'jahshakafx.csv': [0.0],
  'jahshaka.csv': [0.0],
  'selenium.csv': [0.0],
  'paintown.csv': [0.0],
  'h2database.csv': [0.0],
  'x4x.csv': [0.0],
  'elmo.csv': [0],
  'jmoney.csv': [0.0],
  'google-collections.csv': [0.0],
  'baadengine.csv': [0.0],
  'raven-monitoring.csv': [0.0],
  'smbopensoft.csv': [0.0],
  'xharbour.csv': [0.0],
  'flylinkdc.csv': [0.0],
  'reactos-mirror.csv': [0.0],
  'qpe.csv': [0.0],
  'xbrlapi.csv': [0.0],
  'prelude.csv': [0.0],
  'neoengine.csv': [0.0],
  'atunes.csv': [0.0],
  'avifile.csv': [1.0],
  'lamp.csv': [0.0],
  'logicmail.csv': [0.0],
  'wgs-assembler.csv': [0.0],
  'plearn.csv': [0.0],
  'werx.csv': [0.0],
  'plog4u.csv': [0.0],
  'openwbem.csv': [0.0],
  'qgo.csv': [0.0],
  'jaql.csv': [0.0],
  'kmatplot.csv':